In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta

file_path = "dataseta.csv"
data = pd.read_csv(file_path)
data.columns = data.columns.str.strip()
relevant_features = [
    'Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets',
    'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max',
    'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std',
    'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 
    'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 
    'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 
    'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 
    'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 
    'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length',
    'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 
    'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 
    'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 
    'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 
    'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Header Length', 'Fwd Avg Bytes/Bulk', 
    'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 
    'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 
    'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 
    'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 
    'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'Label'
]
flow_data = data[relevant_features]
flow_data['Source IP'] = [f"192.168.1.{random.randint(1, 255)}" for _ in range(len(flow_data))]
flow_data['Destination IP'] = [f"10.0.0.{random.randint(1, 255)}" for _ in range(len(flow_data))]
flow_data['Protocol'] = random.choices(['TCP', 'UDP'], k=len(flow_data))
start_time = datetime.now()
flow_data['Timestamp'] = [start_time + timedelta(milliseconds=i * 100) for i in range(len(flow_data))]
FLOW_TIMEOUT = 120
active_flows = {}

def process_flows(data):
    global active_flows
    for index, row in data.iterrows():
        flow_id = (
            row['Source IP'],
            row['Destination IP'],
            row['Destination Port'],
            row['Protocol']
        )
        current_time = row['Timestamp']
        if flow_id in active_flows:
            active_flows[flow_id]['last_seen'] = current_time
            active_flows[flow_id]['packet_count'] += 1
        else:
            active_flows[flow_id] = {
                'start_time': current_time,
                'last_seen': current_time,
                'packet_count': 1,
                'label': row['Label']
            }
        remove_stale_flows(current_time)

def remove_stale_flows(current_time):
    global active_flows
    for flow_id in list(active_flows.keys()):
        if (current_time - active_flows[flow_id]['last_seen']).total_seconds() > FLOW_TIMEOUT:
            del active_flows[flow_id]

def segment_flows(df, segmentation_rate):
    segmented_data = []
    unique_flows = df.groupby(['Destination Port', 'Label'])
    for _, group in unique_flows:
        total_packets = len(group)
        segment_size = max(1, int(segmentation_rate * total_packets))
        for start in range(0, total_packets, segment_size):
            segment = group.iloc[start:start + segment_size]
            if not segment.empty:
                segmented_data.append(segment)
    return pd.concat(segmented_data, ignore_index=True)

process_flows(flow_data)
print("Active Flows:")
for flow_id, flow_details in active_flows.items():
    print(f"Flow ID: {flow_id}, Details: {flow_details}")
segmentation_rate = 0.25
segmented_dataset = segment_flows(flow_data, segmentation_rate)
output_path = "segmented_dataset_test.csv"
segmented_dataset.to_csv(output_path, index=False)
print(f"\nSegmented dataset saved to {output_path}")
